In [5]:
import requests
import pandas as pd
import time

# Seminar - APIs and real-life coding

## Task 1: Requesting API
### 1a. Create a function requesting data from sreality

```python
base_url = 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&locality_region_id=10&per_page60&page={}'.format(i)

r = requests.get(base_url)
d = r.json()
```

* function should parametrize: 
    * `category_main_cb` - `{'flat':1, 'house':2, 'land':3 }`
    * `category_type_cb` - `{'sell':1,'rent':2}`
    * `locality_region_id` - `{'Praha':10,'Brno':14}`
    * `page` parameter
* use string inputs for `category_main_cb` and `category_type_cb`
* test the validity of inputs
* include try/except clause to handle errors
* function should return JSON data in python types
* do not forget to sleep each request at least 0.5s

In [51]:
def getting_sreality(category_main_cb, category_type_cb, locality_region_id, page):
    try:
        if requests.get('https://www.sreality.cz/api/cs/v2/estates?category_main_cb={}&category_type_cb={}&locality_region_id={}&per_page60&page={}'.format(category_main_cb, category_type_cb, locality_region_id, page)).ok:
            return requests.get('https://www.sreality.cz/api/cs/v2/estates?category_main_cb={}&category_type_cb={}&locality_region_id={}&per_page60&page={}'.format(category_main_cb, category_type_cb, locality_region_id, page)).json()
    except:
        return "the request was unsuccessful"

main_dict = {'flat':1, 'house':2, 'land':3 }
type_dict = {'sell':1,'rent':2}
locality_dict = {'Praha':10,'Brno':14}

def getting_value(inp, dict):
    if inp in dict.keys():
        return dict[inp]
    else:
        try:
            getting_value(input("wrong input, try again: "), dict)
        except:
            return "you have exceeded the number of tries"
            

category_main_cb = getting_value(input("category_main_cb input: "), main_dict)
category_type_cb = getting_value(input("category_type_cb input: "), type_dict)
locality_region_id = getting_value(input("category_main_cb_input: "), locality_dict)
#page = int(input("page input: "))

### 1b. Create a function converting sreality json data into pandas dataframe

In [15]:
def sreality_to_pd(json_data):
    try:
        return pd.read_json(json_data)
    except:
        return json_data

### 1c. link function `1b` into function `1a`

In [104]:
sreality_to_pd(getting_sreality(category_main_cb, category_type_cb, locality_region_id, page))


{'meta_description': '5905 realit v nabídce prodej bytů Praha. Vyberte si novou nemovitost na sreality.cz s hledáním na mapě a velkými náhledy fotografií nabízených bytů.',
 'result_size': 5905,
 '_embedded': {'estates': [{'labelsReleased': [['new_building',
      'not_furnished'],
     []],
    'has_panorama': 0,
    'labels': ['Novostavba', 'Nevybavený'],
    'is_auction': False,
    'labelsAll': [['new_building',
      'personal',
      'balcony',
      'cellar',
      'elevator',
      'parking_lots',
      'garage',
      'not_furnished'],
     ['playground',
      'small_shop',
      'candy_shop',
      'tavern',
      'theater',
      'vet',
      'movies',
      'sightseeing',
      'tram',
      'train',
      'post_office',
      'kindergarten',
      'drugstore',
      'bus_public_transport',
      'school',
      'metro',
      'shop',
      'medic',
      'restaurant',
      'atm',
      'sports']],
    'seo': {'category_main_cb': 1,
     'category_sub_cb': 6,
     'catego

### 1c. Combining multiple requests into single df

* Function should parametrize:
    * `start_page` and `end_page`
    * request parameters
* construct a list of individual request dfs
* then feed it into `pd.concat` function

In [67]:
start_page = int(input("start page input: "))
end_page = int(input("end page input: "))

In [144]:
def single_df(start_page, end_page, category_main_cb, category_type_cb, locality_region_id):
    l = [sreality_to_pd(getting_sreality(category_main_cb, category_type_cb, locality_region_id, i)) for i in range(start_page, end_page + 1)]
    return pd.concat([pd.json_normalize(df["_embedded"]["estates"], max_level = 0) for df in l])



## Task 2: Cleaning data

### 2a. Filter columns
* filter only columns: `['locality', 'price', 'name', 'gps','hash_id','labelsAll','exclusively_at_rk']`
* use `.copy()` to avoid `SettingWithCopyWarning` later


In [171]:
df = single_df(start_page, end_page, category_main_cb, category_type_cb, locality_region_id)[['locality', 'price', 'name', 'gps','hash_id','labelsAll','exclusively_at_rk']].copy(deep = True)

### 2b: GPS
* Convert dictionary in `gps` column into two columns - `lat` and `lon`
* use apply function on gps column
* Note apply can return multiple columns

In [172]:
df = pd.concat([df, df["gps"].apply(pd.Series)], axis = 1).copy(deep = True)

               locality    price                    name  \
0      Praha 3 - Žižkov  6171000  Prodej bytu 3+kk 40 m²   
1         Praha 4 - Krč  4907000   Prodej bytu 3+1 40 m²   
2       Praha 5 - Sobín  3553000  Prodej bytu 2+kk 38 m²   
3      Praha 3 - Žižkov  1995000  Prodej bytu 1+kk 12 m²   
4       Praha 6 - Liboc  4767000   Prodej bytu 3+1 40 m²   
..                  ...      ...                     ...   
16  Praha 3 - Vinohrady  6088000   Prodej bytu 3+1 55 m²   
17    Praha 6 - Břevnov  4201000  Prodej bytu 3+kk 32 m²   
18    Praha 6 - Dejvice  2135000  Prodej bytu 1+kk 10 m²   
19    Praha 6 - Dejvice  3209000  Prodej bytu 2+kk 15 m²   
20  Praha 10 - Hostivař  2684000   Prodej bytu 1+1 29 m²   

                                                  gps     hash_id  \
0   {'lat': 50.07876706597659, 'lon': 14.451852934...  3692241996   
1   {'lat': 50.040967065976595, 'lon': 14.45193193...  1920616268   
2   {'lat': 50.059285065976596, 'lon': 14.27790693...  2225656908   
3  

### 2b. Get flat type from name
* Name is always represented by string `Prodej bytu [type of flat] [Area] m^2`
* try picking third word in string
* check meaningfulness using `.value_counts()`

In [173]:
df["type_of_flat"] = df["name"].apply(lambda v: v.split()[2])

               locality    price                    name  \
0      Praha 3 - Žižkov  6171000  Prodej bytu 3+kk 40 m²   
1         Praha 4 - Krč  4907000   Prodej bytu 3+1 40 m²   
2       Praha 5 - Sobín  3553000  Prodej bytu 2+kk 38 m²   
3      Praha 3 - Žižkov  1995000  Prodej bytu 1+kk 12 m²   
4       Praha 6 - Liboc  4767000   Prodej bytu 3+1 40 m²   
..                  ...      ...                     ...   
16  Praha 3 - Vinohrady  6088000   Prodej bytu 3+1 55 m²   
17    Praha 6 - Břevnov  4201000  Prodej bytu 3+kk 32 m²   
18    Praha 6 - Dejvice  2135000  Prodej bytu 1+kk 10 m²   
19    Praha 6 - Dejvice  3209000  Prodej bytu 2+kk 15 m²   
20  Praha 10 - Hostivař  2684000   Prodej bytu 1+1 29 m²   

                                                  gps     hash_id  \
0   {'lat': 50.07876706597659, 'lon': 14.451852934...  3692241996   
1   {'lat': 50.040967065976595, 'lon': 14.45193193...  1920616268   
2   {'lat': 50.059285065976596, 'lon': 14.27790693...  2225656908   
3  

### 2c. Get area from name
* Naive: select the word before last word
* Then try navigating using the index of `'m²'`
* if this also fail, then you will need to use regex

In [201]:
df["area"] = df["name"].apply(lambda v: int(v.split()[3]))

## Task 3 (Homework): Convert column`labelsAll` into boolean variables

### Task 3a. Get all possible label names
* deal with nested-list structure
* Hint: try sum the whole column to get a nested list of lists.
* Then flatten the nested list (2D to 1D)
* Finally keep only unique elements


In [213]:
df["labelsAll"] = df["labelsAll"].apply(lambda x: [item for sublist in x for item in sublist])

In [220]:
pd.Series([item for sublist in [i for i in df["labelsAll"]] for item in sublist]).unique()

array(['new_building', 'personal', 'balcony', 'cellar', 'elevator',
       'parking_lots', 'garage', 'not_furnished', 'playground', 'vet',
       'small_shop', 'candy_shop', 'tavern', 'theater', 'movies',
       'sightseeing', 'restaurant', 'medic', 'train', 'atm',
       'kindergarten', 'school', 'tram', 'shop', 'sports', 'drugstore',
       'bus_public_transport', 'post_office', 'metro', 'brick', 'loggia',
       'terrace', 'partly_furnished', 'collective', 'panel', 'furnished',
       'in_construction', 'natural_attraction', 'after_reconstruction'],
      dtype=object)

### 4b. Test existence of label `cellar` for offers
* again deal with nested list of list structure
* write generic function `test_existence_of_label(offer_labels,label)`

### 4c. Test existence of all possible labels
* use apply returning series with all labels